In [54]:
#imports:
#install tensorflow, keras, pandas, & scikitlearn.
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ###
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix

#from sklearn import metrics
#import matplotlib.pyplot as plt

In [55]:
data_ANN = pd.read_csv('GoldWeekly2001-2021.csv')

data_ANN = data_ANN.drop('Did_Win', axis=1)
data_ANN = data_ANN.drop('Volume', axis=1)

for i in range(len(data_ANN)):
    if data_ANN.iloc[i,-1] == 0:
        continue
    else:
        data_ANN.iloc[i,-1] = data_ANN.iloc[i,-1] / data_ANN.iloc[i,-2] * 100

data_ANN.head(n=4)

,Date,Open,High,Low,Close,Profit
0,1/1/2001,268.399994,268.399994,267.299988,268.000000,-1.529853
1,1/8/2001,268.000000,268.000000,263.899994,263.899994,0.151570
2,1/15/2001,263.299988,264.299988,263.200012,264.299988,-0.567537
3,1/22/2001,266.399994,266.399994,262.799988,262.799988,1.636232


In [56]:
from sklearn import preprocessing

Label = preprocessing.LabelEncoder()

infoFromLabel = Label.fit_transform(data_ANN['Open'])
#infoFromLabel

In [57]:
#Preprocessing data:
data_labels = data_ANN['Date'].copy()
data_ANN = data_ANN.drop('Date', axis = 1)

#Split data for training and testing
X_train, X_Test_Val = train_test_split(data_ANN, test_size = 0.4, train_size = 0.6, shuffle = False)
X_test, X_val = train_test_split(X_Test_Val, test_size = 0.5, train_size = 0.5, shuffle = False)
#Removing the labels from the data.
train_labels = X_train['Profit'].copy()
X_train = X_train.drop('Profit', axis = 1)

test_labels = X_test['Profit'].copy()
X_test = X_test.drop('Profit', axis = 1)

val_labels = X_val['Profit'].copy()
X_val = X_val.drop('Profit', axis = 1)


In [58]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

#Multi-layer-perceptron
#print(X_train)

arr_train_labels = np.array(train_labels)
arr_test_labels = np.array(test_labels)
arr_val_labels = np.array(val_labels)

X_trainA = []
X_trainB = []
X_trainC = []
X_trainD = []

for i in range(len(X_train)):
    X_trainA.append(X_train.iloc[i,0])
    X_trainB.append(X_train.iloc[i,1])
    X_trainC.append(X_train.iloc[i,2])
    X_trainD.append(X_train.iloc[i,3])

X_trainA = np.array(X_trainA)
X_trainB = np.array(X_trainB)
X_trainC = np.array(X_trainC)
X_trainD = np.array(X_trainD)

X_train_values = [X_trainA, X_trainB, X_trainC, X_trainD]

#Convert Test data to numpy array
X_testA = []
X_testB = []
X_testC = []
X_testD = []

for i in range(len(X_test)):
    X_testA.append(X_test.iloc[i,0])
    X_testB.append(X_test.iloc[i,1])
    X_testC.append(X_test.iloc[i,2])
    X_testD.append(X_test.iloc[i,3])

X_testA = np.array(X_testA)
X_testB = np.array(X_testB)
X_testC = np.array(X_testC)
X_testD = np.array(X_testD)

X_test_values = [X_testA, X_testB, X_testC, X_testD]

#Convert Validation data to numpy array
X_valA = []
X_valB = []
X_valC = []
X_valD = []

for i in range(len(X_val)):
    X_valA.append(X_val.iloc[i,0])
    X_valB.append(X_val.iloc[i,1])
    X_valC.append(X_val.iloc[i,2])
    X_valD.append(X_val.iloc[i,3])

X_valA = np.array(X_valA)
X_valB = np.array(X_valB)
X_valC = np.array(X_valC)
X_valD = np.array(X_valD)

X_val_values = [X_valA, X_valB, X_valC, X_valD]

openPrice = keras.Input(shape=(1, 1), name='Open')
highPrice = keras.Input(shape=(1, 1), name='High')
lowPrice = keras.Input(shape=(1, 1), name='Low')
closePrice = keras.Input(shape=(1, 1), name='Close')

openLayer = LSTM(45, return_sequences=False)(openPrice)
highLayer = LSTM(45, return_sequences=False)(highPrice)
lowLayer = LSTM(45, return_sequences=False)(lowPrice)
closeLayer = LSTM(45, return_sequences=False)(closePrice)

output = keras.layers.concatenate([openLayer, highLayer, lowLayer, closeLayer])

output = Dense(10, activation='linear', name='weightedOutput')(output)

model = keras.Model(inputs = [openPrice, highPrice, lowPrice, closePrice], outputs = [output])
model.compile(optimizer="rmsprop", loss="mae", metrics="mae")

model.fit(X_train_values, arr_train_labels, epochs=25, verbose=2)




# RNN = Sequential()

# #RNN.add.LSTM(LSTM(units=45, activation='relu', return_sequences=False, input_shape = (1, 4)))
# RNN.add.LSTM(units=45, input_shape = (1, 1)) #, activation='relu', return_sequences=False))

# print('2')
# #Dropout regularization is a technique used to avoid overfitting when training neural networks
# RNN.add(Dropout(0.2))
# RNN.add(LSTM(units = 45, return_sequences = True))
# RNN.add(Dropout(0.2))
# RNN.add(LSTM(units = 45, return_sequences = True))
# RNN.add(Dropout(0.2))
# RNN.add(LSTM(units = 45))
# RNN.add(Dropout(0.2))

# RNN.add(Dense(units = 1))
# RNN.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'accuracy')

# #tf.keras.backend.set_floatx('float64')
# #RNN.fit(X_train, train_labels, epochs = 100, batch_size = 32)
# RNN.fit(X_values, arr_train_labels, epochs = 100)

# display(RNN)


# MLP = MLPRegressor(hidden_layer_sizes=(5,5,5), max_iter=20_000)
# MLP.fit(X_train, train_labels)

# results = MLP.predict(X_test)

# result_df = pd.DataFrame(X_test)
# result_df = result_df.assign(Profit = test_labels, Predictions = results)

# display(result_df)

Epoch 1/25
20/20 - 20s - loss: 1.9916 - mae: 1.9916
Epoch 2/25
20/20 - 0s - loss: 1.9631 - mae: 1.9631
Epoch 3/25
20/20 - 0s - loss: 1.9605 - mae: 1.9605
Epoch 4/25
20/20 - 0s - loss: 1.9599 - mae: 1.9599
Epoch 5/25
20/20 - 0s - loss: 1.9638 - mae: 1.9638
Epoch 6/25
20/20 - 0s - loss: 1.9597 - mae: 1.9597
Epoch 7/25
20/20 - 0s - loss: 1.9597 - mae: 1.9597
Epoch 8/25
20/20 - 0s - loss: 1.9604 - mae: 1.9604
Epoch 9/25
20/20 - 0s - loss: 1.9577 - mae: 1.9577
Epoch 10/25
20/20 - 0s - loss: 1.9666 - mae: 1.9666
Epoch 11/25
20/20 - 0s - loss: 1.9616 - mae: 1.9616
Epoch 12/25
20/20 - 0s - loss: 1.9607 - mae: 1.9607
Epoch 13/25
20/20 - 0s - loss: 1.9613 - mae: 1.9613
Epoch 14/25
20/20 - 0s - loss: 1.9552 - mae: 1.9552
Epoch 15/25
20/20 - 0s - loss: 1.9610 - mae: 1.9610
Epoch 16/25
20/20 - 0s - loss: 1.9585 - mae: 1.9585
Epoch 17/25
20/20 - 0s - loss: 1.9584 - mae: 1.9584
Epoch 18/25
20/20 - 0s - loss: 1.9591 - mae: 1.9591
Epoch 19/25
20/20 - 0s - loss: 1.9579 - mae: 1.9579
Epoch 20/25
20/20 - 

In [59]:
#currentMoney = 1000

#for ((length of testing data)-1)
#    if predicting_a_win:
#        currentMoney = currentMoney * (Profit[i] + 1)
#    
#print(CurrentMoney) #then do this for validation.
#
#
#
